In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import os

2025-09-13 12:39:21.654365: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-13 12:39:21.716304: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-13 12:39:21.717495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-13 12:39:22.634973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# MENGAMBIL GESTUR 2 TANGAN

In [1]:
import cv2
import os
import time
import math
import numpy as np
from cvzone.HandTrackingModule import HandDetector

# --- PENGATURAN & KONFIGURASI ---
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2, detectionCon=0.8, minTrackCon=0.7)

# Parameter untuk cropping dan penyimpanan
offset = 20
imgSize = 300
folder = "datasets/F" # Ganti nama gestur Anda
os.makedirs(folder, exist_ok=True)

# --- VARIABEL UNTUK PENGAMBILAN GAMBAR OTOMATIS ---
max_images = 1000
capture_interval = 0.1 # Jeda antar pengambilan gambar (setelah stabil)
last_capture_time = 0

# --- VARIABEL BARU UNTUK CEK STABILITAS ---
stability_delay = 2.0       # Harus stabil selama 2 detik
stability_threshold = 30    # Ambang batas toleransi gerakan (makin kecil makin ketat)
stable_start_time = None    # Waktu saat gestur mulai stabil
last_bbox = None            # Menyimpan posisi bounding box terakhir

# --- PROGRAM UTAMA ---
while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False) # draw=False agar bersih

    # Dapatkan jumlah gambar saat ini di folder
    count = len(os.listdir(folder))
    status_text = "" # Teks untuk menampilkan status di layar

    if len(hands) == 2:
        # Ambil bbox kedua tangan dan hitung container gabungan
        x1, y1, w1, h1 = hands[0]['bbox']
        x2, y2, w2, h2 = hands[1]['bbox']
        x_min = max(0, min(x1, x2) - offset)
        y_min = max(0, min(y1, y2) - offset)
        x_max = min(img.shape[1], max(x1 + w1, x2 + w2) + offset)
        y_max = min(img.shape[0], max(y1 + h1, y2 + h2) + offset)

        # Gambar kotak container di tampilan utama
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        
        # --- LOGIKA CEK STABILITAS ---
        current_bbox = (x_min, y_min, x_max, y_max)
        
        if last_bbox is not None:
            # Hitung perbedaan posisi bbox saat ini dan sebelumnya
            diff = np.abs(np.array(current_bbox) - np.array(last_bbox)).sum()

            if diff < stability_threshold: # Jika gestur dianggap stabil
                if stable_start_time is None:
                    # Mulai timer jika ini pertama kali stabil
                    stable_start_time = time.time()
                
                elapsed_time = time.time() - stable_start_time
                status_text = f"Stabil: {elapsed_time:.1f}s"
                
                # --- LOGIKA PENYIMPANAN GAMBAR (SETELAH STABIL) ---
                if elapsed_time >= stability_delay:
                    status_text = ">> MENYIMPAN GAMBAR <<"
                    
                    if count < max_images:
                        current_time = time.time()
                        if current_time - last_capture_time > capture_interval:
                            last_capture_time = current_time
                            
                            # Crop gambar
                            imgCrop = img[y_min:y_max, x_min:x_max]
                            if imgCrop.size == 0: continue

                            # Normalisasi ukuran gambar
                            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                            h_crop, w_crop, _ = imgCrop.shape
                            aspectRatio = h_crop / w_crop

                            if aspectRatio > 1:
                                k = imgSize / h_crop
                                wCal = math.ceil(k * w_crop)
                                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                                wGap = math.ceil((imgSize - wCal) / 2)
                                imgWhite[:, wGap:wCal + wGap] = imgResize
                            else:
                                k = imgSize / w_crop
                                hCal = math.ceil(k * h_crop)
                                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                                hGap = math.ceil((imgSize - hCal) / 2)
                                imgWhite[hGap:hCal + hGap, :] = imgResize

                            # Simpan gambar
                            save_path = f"{folder}/Image_{count + 1}.jpg"
                            cv2.imwrite(save_path, imgWhite)
                            print(f"Gambar disimpan: {save_path}")
                            cv2.imshow("Gestur Tersimpan", imgWhite)
            else:
                # Jika gestur bergerak, reset timer
                stable_start_time = None
                status_text = "Gestur Bergerak, Timer Reset"
        
        # Update posisi bbox terakhir
        last_bbox = current_bbox
    else:
        # Jika tangan tidak terdeteksi, reset semuanya
        stable_start_time = None
        last_bbox = None
        status_text = "Tangan tidak terdeteksi"

    # --- Tampilkan feedback di layar ---
    if count >= max_images:
        feedback_text = "SELESAI"
        cv2.putText(img, feedback_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    else:
        feedback_text = f"Tersimpan: {count}/{max_images}"
        cv2.putText(img, feedback_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.putText(img, status_text, (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.imshow("Kamera", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2025-09-13 15:22:42.837565: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-13 15:22:43.018619: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-13 15:22:43.020662: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-13 15:22:44.135847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0000 00:00:1757748165.612638  195527 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1757748165.623178  195637 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 

Gambar disimpan: datasets/F/Image_1.jpg
Gambar disimpan: datasets/F/Image_2.jpg
Gambar disimpan: datasets/F/Image_3.jpg
Gambar disimpan: datasets/F/Image_4.jpg
Gambar disimpan: datasets/F/Image_5.jpg
Gambar disimpan: datasets/F/Image_6.jpg
Gambar disimpan: datasets/F/Image_7.jpg
Gambar disimpan: datasets/F/Image_8.jpg
Gambar disimpan: datasets/F/Image_9.jpg
Gambar disimpan: datasets/F/Image_10.jpg
Gambar disimpan: datasets/F/Image_11.jpg
Gambar disimpan: datasets/F/Image_12.jpg
Gambar disimpan: datasets/F/Image_13.jpg
Gambar disimpan: datasets/F/Image_14.jpg
Gambar disimpan: datasets/F/Image_15.jpg
Gambar disimpan: datasets/F/Image_16.jpg
Gambar disimpan: datasets/F/Image_17.jpg
Gambar disimpan: datasets/F/Image_18.jpg
Gambar disimpan: datasets/F/Image_19.jpg
Gambar disimpan: datasets/F/Image_20.jpg
Gambar disimpan: datasets/F/Image_21.jpg
Gambar disimpan: datasets/F/Image_22.jpg
Gambar disimpan: datasets/F/Image_23.jpg
Gambar disimpan: datasets/F/Image_24.jpg
Gambar disimpan: datasets

# MENGAMBIL GESTUR 1 TANGAN

In [ ]:
import cv2
import os
import time
import math
import numpy as np
from cvzone.HandTrackingModule import HandDetector

# --- PENGATURAN & KONFIGURASI ---
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1, detectionCon=0.8, minTrackCon=0.7)

offset = 20
imgSize = 300
folder = "datasets/E"   # Ganti sesuai huruf/gestur
os.makedirs(folder, exist_ok=True)

# --- VARIABEL PENGAMBILAN GAMBAR ---
max_images = 1000
capture_interval = 0.1  # ambil gambar setiap 0.1s setelah stabil
last_capture_time = 0

# --- VARIABEL CEK STABILITAS ---
stability_delay = 2.0       # butuh stabil 2 detik
stability_threshold = 30
stable_start_time = None
last_bbox = None

# --- PROGRAM UTAMA ---
while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)  # mirror
    hands, img = detector.findHands(img, flipType=False)

    count = len(os.listdir(folder))  # hitung jumlah file tersimpan
    status_text = ""

    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        # bounding box dengan offset
        x_min = max(0, x - offset)
        y_min = max(0, y - offset)
        x_max = min(img.shape[1], x + w + offset)
        y_max = min(img.shape[0], y + h + offset)

        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

        # cek stabilitas
        current_bbox = (x_min, y_min, x_max, y_max)
        if last_bbox is not None:
            diff = np.abs(np.array(current_bbox) - np.array(last_bbox)).sum()

            if diff < stability_threshold:
                if stable_start_time is None:
                    stable_start_time = time.time()

                elapsed_time = time.time() - stable_start_time
                status_text = f"Stabil: {elapsed_time:.1f}s"

                # mulai simpan setelah stabil 2 detik
                if elapsed_time >= stability_delay and count < max_images:
                    current_time = time.time()
                    if current_time - last_capture_time > capture_interval:
                        last_capture_time = current_time

                        imgCrop = img[y_min:y_max, x_min:x_max]
                        if imgCrop.size == 0:
                            continue

                        # canvas putih
                        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                        h_crop, w_crop, _ = imgCrop.shape
                        aspectRatio = h_crop / w_crop

                        if aspectRatio > 1:
                            k = imgSize / h_crop
                            wCal = math.ceil(k * w_crop)
                            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                            wGap = math.ceil((imgSize - wCal) / 2)
                            imgWhite[:, wGap:wCal + wGap] = imgResize
                        else:
                            k = imgSize / w_crop
                            hCal = math.ceil(k * h_crop)
                            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                            hGap = math.ceil((imgSize - hCal) / 2)
                            imgWhite[hGap:hCal + hGap, :] = imgResize

                        save_path = f"{folder}/Image_{count + 1}.jpg"
                        cv2.imwrite(save_path, imgWhite)
                        print(f"Gambar disimpan: {save_path}")

                        # tampilkan hasil gambar tersimpan dengan info jumlah
                        cv2.putText(imgWhite, f"Total: {count+1}", (10, 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        cv2.imshow("Gestur Tersimpan", imgWhite)
            else:
                stable_start_time = None
                status_text = "Gestur Bergerak, Timer Reset"

        last_bbox = current_bbox
    else:
        stable_start_time = None
        last_bbox = None
        status_text = "Tangan tidak terdeteksi"

    # tampilkan jumlah data di layar utama
    if count >= max_images:
        feedback_text = "PENGAMBILAN SELESAI"
        cv2.putText(img, feedback_text, (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    else:
        feedback_text = f"Tersimpan: {count}/{max_images}"
        cv2.putText(img, feedback_text, (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.putText(img, status_text, (50, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.imshow("Kamera Pengambilan Gestur (1 Tangan)", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1757747905.916730  194318 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1757747905.919842  195223 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-29-generic)


Gambar disimpan: datasets/E/Image_1.jpg
Gambar disimpan: datasets/E/Image_2.jpg
Gambar disimpan: datasets/E/Image_3.jpg
Gambar disimpan: datasets/E/Image_4.jpg
Gambar disimpan: datasets/E/Image_5.jpg
Gambar disimpan: datasets/E/Image_6.jpg
Gambar disimpan: datasets/E/Image_7.jpg
Gambar disimpan: datasets/E/Image_8.jpg
Gambar disimpan: datasets/E/Image_9.jpg
Gambar disimpan: datasets/E/Image_10.jpg
Gambar disimpan: datasets/E/Image_11.jpg
Gambar disimpan: datasets/E/Image_12.jpg
Gambar disimpan: datasets/E/Image_13.jpg
Gambar disimpan: datasets/E/Image_14.jpg
Gambar disimpan: datasets/E/Image_15.jpg
Gambar disimpan: datasets/E/Image_16.jpg
Gambar disimpan: datasets/E/Image_17.jpg
Gambar disimpan: datasets/E/Image_18.jpg
Gambar disimpan: datasets/E/Image_19.jpg
Gambar disimpan: datasets/E/Image_20.jpg
Gambar disimpan: datasets/E/Image_21.jpg
Gambar disimpan: datasets/E/Image_22.jpg
Gambar disimpan: datasets/E/Image_23.jpg
Gambar disimpan: datasets/E/Image_24.jpg
Gambar disimpan: datasets